# SingularityDAO: Automating the airdrop calculations

## Overview

1. **Take Snapshots** Generate snapshots placed at random intervals (and optionally store results in a DB)
2. **Calculate Eligibility**  Categorize each address and apply custom logic depending on the airdrop's eligibility criteria 
3. **Serve To The Portal** Store eligibility of each type of airdrop per user in a DB and make it accessible to the portal

## 1. Take Snapshots

### Using a database and an ethereum node

A relational DB can be used to store a list of balances. I think it's what could be currently implemented in the existing airdrop.

The balances in the database could be bootstrapped by an initial import from Etherscan or by the initial snapshot, and be kept up to date by running a node that listens for new transactions.

## 2. Calculate Eligibility

The following criteria follows the airdrop mechanics as described in the [official post](https://medium.com/singularitydao/singularitydao-updated-airdrop-eb6fc9d2210b)

The airdrop mechanics encompass 3 different groups:
- Stakers
- Holders
- Liquidity Providers

The eligibility and distribution of each category are calculated separately.

### Stakers

#### Eligibility

For stakers, the snapshots correspond directly to the staking windows, which happen monthly.

To calculate the eligibility of a staker, the main criteria is whether they were part of the snapshot on the 17th of April 2021, at 23:59 UTC+0.

A staker it's eligible if they're staking on all the snapshots taken.

#### Distribution

2,000,000 SDAO tokens are distributed among the stakers:
- 1,000,000 distributed proportionally to the eligible wallets that claim the airdrop, without taking into account stake, only number of participants
- 1,000,000 distributed proportionally to the amount staked

### Holders and Liquidity Providers

Holders and Liquidity Providers are rewarded by the following formula:
`Reward = total_reward * log10(1+user_balance) / SUM(log10(1+user_balance))`

#### Eligility

There are two sets of random snapshots taken, one for Holders and another one for Liquidity Providers.

My only question here it's whether the liquidity providers eligibility it's calculated by their actual AGIX in Uniswap, or by converting the ETH & AGIX balances to fiat and calculating the AGIX that it's worth at the market price of when the snapshot was taken.

#### Distribution

In the two remaining months, July and August, 1,800,000 SDAO are left to be distributed. 825,000 in July and 975,000 in August.

Once Liquidity Providers and Holders pass their respective eligibility criterio, they are added to the same pool of tokens.

The reward for each user it's calculated by the ratio of the user's balance to the total balance of all the eligible users, smoothing both values with the `log10` function.
Finally, the final reward it's the result of multiplying the previous value by the total reward for that month.

## 3. Serve To The Portal

Once the eligibility criteria of all users has been processed, the results can be imported into a DB to be served to the portal using GraphQL on the client side.

# Final Notes

## Distributing the Airdrop

To the best of my knowledge, there are two ways in which claiming the airdrod could be accomplished in the current system.
In both ways, I think that the server has to have a private key to sign a message, which approves X amount for the eligible wallet.

- **On-chain:** The eligible wallets are registered on-chain using the server/admin's private key. The gas cost is payed by the server and the user.
- **Off-chain:** the server signs a message on the fly when the user claims the airdrop. The user then appends the message to the claiming transaction. The gas cost is payed only by the user.

## E2E Testing

Once the processing of the airdrop has been automated with its own pipeline and the final eligibility of all addresses is in the DB, what remains is to test that all the other systems work as expected.

For that, the E2E testing tool would be used (currently WIP).

[You can find out more about the E2E testing tool here](https://github.com/mathisbeauty/e2e-tester)
